In [3]:
import networkx as nx
from random import *
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import ast
import numpy as np


### Import tweeter dataset

In [6]:
df = pd.read_csv("emotion_tweets_cleaned.csv")

### Create conections


In [7]:
inter = {}
connections = {}

x = df.sample(15000)

for i,r in x.iterrows():
    for pair in itertools.permutations(ast.literal_eval(r.hashtags),2):
        
        pair = sorted(pair)
        
        key = pair[0]+'&&'+pair[1]
        
        if key in inter:
            inter[key]['i'] =inter[key]['i'] + 1
            
            if r.emotion in inter[key]['emotion']:
                 inter[key]['emotion'][r.emotion] = inter[key]['emotion'][r.emotion] + 1
            else:
                inter[key]['emotion'][r.emotion] = 1
            
            break
        else:
            inter[key] = {'i':1, 'emotion':{r.emotion:1}}
        
        if pair[0] in connections:
            connections[pair[0]] = connections[pair[0]] + 1
        else:
            connections[pair[0]] = 1
            
        if pair[1] in connections:
            connections[pair[1]] = connections[pair[1]] + 1
        else:
            connections[pair[1]] = 1
        
        
new_inter = {}
for item in inter:
    if  inter[item]['i'] > 1:
        new_inter[item] = inter[item]
        
        
# Get individual tags that were selected
selHash = []
for item in new_inter:
    pair1 = item.split('&&')[0]
    pair2 = item.split('&&')[1]
    
    if pair1 not in selHash: 
        selHash += [pair1]
    if pair2 not in selHash: 
        selHash += [pair2]
        

df['t_interactions'] = df.nlikes + df.nreplies + df.nretweets

interactions = []
for i,r in df[df.t_interactions>0].iterrows():
    hashs = ast.literal_eval(r.hashtags)
    hashs = [(x,r.t_interactions) for x in  hashs]
    
    interactions += hashs
    
inter = pd.DataFrame(interactions,columns = ['tag','n']).groupby('tag',as_index =False).sum()    
inter = dict(zip(inter.tag, inter.n))
        

# Get which emotion is more relevant for each sentiment
emot = []
for i,r in df.iterrows():
    hashs = ast.literal_eval(r.hashtags)
    hashs = [(x,r.emotion,1) for x in  hashs]
    emot += hashs
    
emotions = pd.DataFrame(emot,columns = ['tag','emotion','n']).groupby(['tag','emotion'],as_index =False).count()  
emotions = emotions.sort_values('n', ascending=False).drop_duplicates(['tag'])

emotions_count = {}
for i,r in emotions.iterrows():
    emotions_count[r.tag] = {'emotion':r.emotion, 'n':r.n}    

### Plot connections

In [ ]:
G = nx.Graph()

plt.figure(3,figsize=(23,12)) 

for i in new_inter:
    
    G.add_edge(i.split('&&')[0],i.split('&&')[1], weight=new_inter[i]['i']/7)
    
colors  = {'joy':'tab:red','anger':'tab:blue','fear':'tab:green','trust':'tab:orange'}


llaves = [x for x in G.nodes().keys() if G.degree[x] >100]
llaves = [x for x in G.nodes().keys() ]


included = []
for edge in G.edges():
    if len([i for i in list(edge) if i in llaves]) > 0:
        included += list(edge)
        
llaves = pd.Series(llaves + included).unique().tolist()

keys = list(G.nodes().keys())
a= [ G.remove_node(key) for key in keys if key not in llaves]

    
weights = [0.1 for i in new_inter]
sizes = [ (inter[node]/connections[node] + 30)**1.25  if node in inter and connections[node] > 6 else 10 for node in G.nodes()]
color_map =  [ colors[emotions_count[node]['emotion']] if node in emotions_count else 'tab:gray' for node in G.nodes()]


alpha_map = [ emotions_count[node]['n'] if node in emotions_count else 1 for node in G.nodes()]
alpha_map = [ 0.05 + np.log(i)/13 for i in alpha_map]


seeds =  ['#thankful','#feelinggood','#blessed','#happy','#estatic','#goodmood','#yay','#worstfear','#fear','#frightened',
          '#worstfear','#fear','#frightened','#horror','#dismay','#anxious','#afraid',
          '#horror','#dismay','#anxious','#afraid','#disappointed', '#angry','#mad','#pissed','#furious','#hate','#angrytweet'] 


seed_labels = {}
new_labels = {}

for node in G.degree:
    if node[1]>10: 
        if node[0] in seeds:
            seed_labels[node[0]] = node[0] 
        else:
            new_labels[node[0]] = node[0] 
            
popularity = {}
for node in G.degree:
    if (node[0] in inter):
        if (inter[node[0]]/connections[node[0]] >25 and (connections[node[0]] > 10)) :
            if (node[0] not in seeds ):
                popularity[node[0]] = node[0] 
        
popularity = {}
for node in G.degree:
    if (node[0] in inter):
        if (inter[node[0]]/connections[node[0]] >25 and (connections[node[0]] > 10)) :
            if (node[0] not in seeds ):
                popularity[node[0]] = node[0] 
                
                
pos = nx.spring_layout(G)

all = {}
_ = [all.update({node[0]:node[0]})  for node in G.degree]

nx.draw(G,  node_size = sizes,width=weights,node_color=color_map, alpha=0.70,with_labels=False, pos=pos)
nx.draw_networkx_labels(G,pos, seed_labels,font_size=15,font_color='black',font_family='Tahoma',alpha=0.85,font_weight='bold');
nx.draw_networkx_labels(G,pos, new_labels,font_size=19, font_color='darkred', family='Arial',alpha=0.95);
nx.draw_networkx_labels(G,pos, popularity,font_size=19, font_color='darkblue', family='Arial',alpha=0.95);
# nx.draw_networkx_labels(G,pos,all,font_size=19, font_color='darkblue', family='Arial',alpha=0.95);
